<a href="https://colab.research.google.com/github/teticio/aventuras-en-texto/blob/master/Amazon_Opiniones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Descargar los datos

In [0]:
import pickle

!wget -q -O train_text.p 'https://docs.google.com/uc?export=download&id=1-0WEtyIuvB8pxENu8WJvgYFb8aMc4F8R'
!wget -q -O train_label.p 'https://docs.google.com/uc?export=download&id=1-AKY4RWMXDg035vTJQLqUXIjYMw6Ja1t'
!wget -q -O test_text.p 'https://docs.google.com/uc?export=download&id=1-4WBoOczIwOpd-Qu5n4dINcor9wC7UUk'

train_text = pickle.load(open('train_text.p', 'rb'))
train_label = pickle.load(open('train_label.p', 'rb'))
test_text = pickle.load(open('test_text.p', 'rb'))

### -----8<-----8<-----8<-----8<-----8<-----8<-----8<-----8<-----8<-----8<-----8<

## Tu modelo va aquí

In [0]:
import re

max_len = 512
train_text = [re.sub(r'([^\s\w]|_)+', ' ', _).lower().split() for _ in train_text]
train_text = [_[:max_len] + [''] * (max_len - len(_)) for _ in train_text]

In [9]:
import gensim
from gensim.models.callbacks import CallbackAny2Vec

class logger(CallbackAny2Vec):
    def __init__(self):
        print('Starting...')
        self.epoch = 0
        self.loss = 0

    def on_epoch_end(self, model):
        print('#{}'.format(self.epoch), 'loss =',
              (model.get_latest_training_loss() - self.loss) / model.batch_words)
        self.epoch += 1
        self.loss = model.get_latest_training_loss()

embedding_model = gensim.models.Word2Vec(sentences=train_text, size=100, min_count=0, iter=15, compute_loss=True, callbacks=[logger()])
print(embedding_model)

Starting...
#0 loss = 14.5525484375
#1 loss = 11.6133125
#2 loss = 10.8104078125
#3 loss = 10.4901625
#4 loss = 9.43931875
#5 loss = 9.334275
#6 loss = 9.31493125
#7 loss = 9.2712625
#8 loss = 9.14311875
#9 loss = 8.9786
#10 loss = 7.8565625
#11 loss = 8.625075
#12 loss = 8.33815
#13 loss = 8.3579875
#14 loss = 8.5798125
Word2Vec(vocab=9005, size=100, alpha=0.025)


In [0]:
import numpy as np

X_train = np.concatenate([np.array([embedding_model.wv.vocab[word].index for word in _])[np.newaxis, :] for _ in train_text], axis=0)
y_train = np.array(train_label)

In [12]:
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense
from keras.layers.embeddings import Embedding

model = Sequential()
model.add(Embedding(
    input_dim=embedding_model.wv.vectors.shape[0],
    output_dim=embedding_model.wv.vectors.shape[1],
    input_length=len(X_train[0]),
    weights=[embedding_model.wv.vectors],
    trainable=False))
model.add(Conv1D(256, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(256, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(256, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Using TensorFlow backend.
W0717 14:24:05.824485 140450420889472 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0717 14:24:05.846405 140450420889472 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0717 14:24:05.849318 140450420889472 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0717 14:24:05.866328 140450420889472 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0717 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 512, 100)          900500    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 508, 256)          128256    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 101, 256)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 97, 256)           327936    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 19, 256)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 15, 256)           327936    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 256)               0         
__________

In [13]:
#model.load_weights('AmazonModel.h5')
callbacks = [EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='min'),
             ModelCheckpoint('AmazonModel.h5', save_best_only=True, monitor='val_loss', mode='min')]
model.fit(X_train, 
          y_train,
          validation_split=0.1,
          epochs=100,
          batch_size=512,
          callbacks=callbacks)

Train on 4491 samples, validate on 499 samples
Epoch 1/100
4491/4491 [==============================] - 4s 965us/step - loss: 0.5662 - acc: 0.7715 - val_loss: 0.4410 - val_acc: 0.8257
Epoch 2/100
4491/4491 [==============================] - 2s 368us/step - loss: 0.4062 - acc: 0.8521 - val_loss: 0.4497 - val_acc: 0.8257
Epoch 3/100
4491/4491 [==============================] - 2s 368us/step - loss: 0.3885 - acc: 0.8521 - val_loss: 0.4021 - val_acc: 0.8257
Epoch 4/100
4491/4491 [==============================] - 2s 370us/step - loss: 0.3593 - acc: 0.8530 - val_loss: 0.3835 - val_acc: 0.8257
Epoch 5/100
4491/4491 [==============================] - 2s 368us/step - loss: 0.3237 - acc: 0.8617 - val_loss: 0.3330 - val_acc: 0.8397
Epoch 6/100
4491/4491 [==============================] - 2s 370us/step - loss: 0.2972 - acc: 0.8651 - val_loss: 0.3086 - val_acc: 0.8657
Epoch 7/100
4491/4491 [==============================] - 2s 368us/step - loss: 0.2784 - acc: 0.8751 - val_loss: 0.3093 - val_acc: 0

In [0]:
test_text = [re.sub(r'([^\s\w]|_)+', '', _).lower().split() for _ in test_text]
test_text = [_[:max_len] + [''] * (max_len - len(_)) for _ in test_text]

X_test = np.concatenate([np.array([embedding_model.wv.vocab.get(word,embedding_model.wv.vocab['']).index for word in _])[np.newaxis, :] for _ in test_text], axis=0)

model.load_weights('AmazonModel.h5')
test_pred = model.predict(X_test)

In [62]:
from google.colab import drive
drive.mount('/content/drive')

pickle.dump(test_pred, open('/content/drive/My Drive/test_pred.p', 'wb'))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### -----8<-----8<-----8<-----8<-----8<-----8<-----8<-----8<-----8<-----8<-----8<

## Tus predicciones deberían tener este formato

- Un numpy array de 2736 probabilidades de que la etiqueta sea '1' por cada uno de las muestras de test_text
- El que menor log_loss (binary_corss_entropy) consigue gana el premio y entra en la liga
- Manda el fichero pickleado a teticio@gmail.com para obtener tu puntuación

In [69]:
test_pred = pickle.load(open('/content/drive/My Drive/test_pred.p', 'rb'))
print(len(test_pred))
test_pred

2736


array([[0.8434883 ],
       [0.89538974],
       [0.13218054],
       ...,
       [0.7121408 ],
       [0.6178071 ],
       [0.979002  ]], dtype=float32)